In [1]:
import tensorflow as tf
import transformers
import data
from keras import backend as K
import pandas as pd
import re
from nltk.corpus import  stopwords
from mutator import mutant

2024-12-02 20:18:01.587931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /lib:/lib/
2024-12-02 20:18:01.587946: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv("data/mydata/twitter_reddit_merge.csv")[['text', 'sarcastic']]

In [8]:
df.shape

(20922, 2)

In [9]:
mu = mutant.TextMutant()

In [10]:
# mu.remove_words()
part_1 = df.sample(frac = 0.7).dropna()
part_2 = df.drop(part_1.index).dropna()

syn = part_1
rem = part_2.sample(frac = 0.5)
shuf = part_2.drop(rem.index)

syn['text'] = syn['text'].apply(mu.replace_words)
rem['text'] = rem['text'].apply(mu.remove_words)
shuf['text'] = shuf['text'].apply(mu.shuffle_words)

In [11]:
df = pd.concat([df, syn, rem, shuf],axis=0, ignore_index=True)

In [12]:
df.shape

(41838, 2)

In [13]:
df.to_csv("data/mydata/mutated_twitter_reddit.csv")

In [52]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = transformers.RobertaTokenizer.from_pretrained(model_name)

In [71]:
from sklearn.utils import resample

minority_class = df[df['sarcastic'] == 1]
majority_class = df[df['sarcastic'] == 0]
minority_upsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)

# %%
df = pd.concat([majority_class, minority_upsampled])


# %%
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['sarcastic'], stratify=df['sarcastic'], test_size=0.2, random_state=42)

In [69]:
def tokenize_function(text: list):
    return tokenizer(text, max_length=128, padding="max_length", truncation=True, return_tensors="tf")

In [65]:
# x_train.values.tolist()

In [70]:
train_enc = tokenize_function(x_train.astype(str).values.tolist())
test_enc = tokenize_function(x_test.astype(str).values.tolist())

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_enc), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_enc), y_test))

batch_size = 16
train_dataset = train_dataset.shuffle(len(y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(len(y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

<class 'pandas.core.series.Series'>


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# %%


# %%
config = transformers.BertConfig.from_pretrained(model_name,
                                    output_hidden_states=True)
roberta_model = transformers.TFRobertaModel.from_pretrained(model_name, config=config)
input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
roberta_outputs = roberta_model(input_ids=input_ids, attention_mask=attention_mask)

x1 = tf.keras.layers.Dropout(0.1)(roberta_outputs.last_hidden_state)
x1 = tf.keras.layers.Conv1D(filters=128, kernel_size=3,padding='same')(x1)
x1 = tf.keras.layers.LeakyReLU()(x1)
x1 = tf.keras.layers.GlobalMaxPooling1D()(x1)
output = tf.keras.layers.Dense(1, activation="softmax")(x1)
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
print(model.summary())

# %%
def f1_score(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Threshold predictions
    tp = K.sum(y_true * y_pred)  # True positives
    fp = K.sum((1 - y_true) * y_pred)  # False positives
    fn = K.sum(y_true * (1 - y_pred))  # False negatives

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-8),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(), 
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.BinaryCrossentropy(),
        f1_score
    ]
)


# %%
# model.fit(dataset, epochs=5)
out = model.fit(train_dataset, epochs=5)
results = model.evaluate(test_dataset, batch_size=128)
print(out.history)
print("eval")
print(results)

import json
with open("results/roberta_cnn_mutated.json", "w+") as file:
    file.write(json.dumps(out.history))

model.save("output/sarc_detect_mutated.keras")

<class 'list'>


ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.